In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import KY_utils.KY_baseline_functions as base

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['general_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['general_two_year'].values

### Model

In [3]:
#### Logistic
c = [1e-4, 1e-3, 1e-2, 1e-1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 1e-2, 1e-1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 1e-2, 1e-1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [100,150,200]
depth = [7,8,9]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y, 
                     depth=depth, 
                     estimators=n_estimators,
                     seed=816)

#### XGBoost
learning_rate = [0.1]
depth = [4,5,6]
n_estimators = [100,150]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_general2_ky_model = {"Logistic": logistic_summary,
                             "Lasso": lasso_summary,
                             "LinearSVM": svm_summary,
                             "RF": rf_summary,
                             "XGBoost": xgb_summary}

In [4]:
results = []
for model_name, model_summary in summary_general2_ky_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['FL_score']])
results

[['Logistic', 0.7299389757737996, 0.0002872717471843478, 0.6005418336792407],
 ['Lasso', 0.7299209870525777, 0.00028855338714195433, 0.6007953231198213],
 ['LinearSVM', 0.7293576007577794, 0.0002788690874753197, 0.6001341381623071],
 ['RF', 0.7412350968429621, 0.012859442210436556, 0.615715289109988],
 ['XGBoost', 0.7465434105503863, 0.012496192856098931, 0.5975855130784709]]

In [6]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\baselines\\Two Year\\"
results = [["", "Logistic", "Lasso", "Linear SVM", "Random Forest", "XGBoost"],
           ["General",             
            round(logistic_summary['FL_score'],3), 
            round(lasso_summary['FL_score'], 3), 
            round(svm_summary['FL_score'],3), 
            round(rf_summary['FL_score'],3), 
            round(xgb_summary['FL_score'],3)]]
with open(path + 'Two Year KY_model.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)